# Set up and global variables

In [ ]:
from pathlib import Path
from collections import defaultdict

import os
import matplotlib.pyplot as plt
import re
import seaborn as sns
import pandas as pd
import numpy as np
import random
import json
from matplotlib.colors import ListedColormap
from IPython.display import display, HTML
from tqdm import tqdm
from scipy.stats import spearmanr

import src.ipython_loader as loader
from src.prioritization import *

RESOLUTION = 300
VERSION = '0.0.0'
DATASET_PATH = Path('data') / 'datasets' / f'ipython_{VERSION}'
OUTPUT_PATH = DATASET_PATH / 'heuristics'
BINARY_CMAP = ListedColormap(['red', 'green'])

IMAGE_DIR = Path('images') / "heuristics"

os.makedirs(OUTPUT_PATH, exist_ok=True)
os.makedirs(IMAGE_DIR, exist_ok=True)

MIN_TASK_DEFECT_SUBMISSIONS = 5

## Utils

In [ ]:
def truncate_text(text: str, max_length: int=20):
    """Truncate text if it exceeds the maximum length."""
    if len(text) > max_length:
        return text[:max_length - 3] + '...'
    return text

def abbreviate_and_truncate_text(text: str, ordered_abbreviations: dict | None=None, max_length: int=20):
    """Shorten text by applying a list of abbreviations and truncating if necessary."""
    current_text = text

    if ordered_abbreviations:
        for full_word, abbr in ordered_abbreviations.items():
            # Use regex with word boundaries to ensure we replace full words only
            pattern = re.escape(full_word)
            current_text = re.sub(pattern, abbr, current_text, flags=re.IGNORECASE)

            if len(current_text) <= max_length:
                return current_text
    
    return truncate_text(current_text)

ordered_abbreviations = {
    'whitespace': 'ws',
    'constant': 'const',
    'variable': 'var',
    'function': 'func',
    'parameter': 'param',
    'expression': 'expr',
    'argument': 'arg',
    'operator': 'op',
    'operation': 'op',
    'augmentable': 'aug',
    'assignment': 'assign',
    'container': 'cont',
    'statement': 'stmt',
    'arithmetic': 'arith',
    'condition': 'cond',
    'identifier': 'identif',
    'multiple': 'multi',
    'redundant': 'redun',
    'necessary': 'necces',
    'comparison': 'compar',
    'negated': 'neg',
    'unreachable': 'unreach',
    'inappropriate': 'inapp',
    'parenthesis': '()',
}

In [ ]:
def task_and_defect_description(task, defect, items, defects, log, defect_log):
    """Generate an HTML display for a specific task and defect."""
    task_row = items.loc[task]
    defect_row = defects.loc[defect]
    submissions = log[(log["item"] == task) & (defect_log[defect])]
    
    return f"""
    <div style="display: flex; justify-content: space-between; gap: 20px;">
        
        <div style="width: 48%; border: 1px solid #ccc; padding: 10px; border-radius: 5px;">
            <h3>{task_row["name"]}</h3>
            <div><strong>Instructions:</strong><br>{task_row["instructions"]}</div>
            <div><strong>Solution:</strong><br>
                <pre style="background-color: #2e2e2e; color: #f5f5f5; padding: 10px; border-radius: 5px; font-family: monospace;">{task_row["solution"]}</pre>
            </div>
        </div>
        
        
        <div style="width: 48%; border: 1px solid #ccc; padding: 10px; border-radius: 5px;">
            <h3>{defect_row["defect name"]}</h3>
            <div><strong>Defect Type:</strong> {defect_row["defect type"]}</div>
            <div><strong>Severity:</strong> {defect_row["severity"]}</div>
            <div><strong>Description:</strong><br>{defect_row["description"]}</div>
            
            <div style="display: flex; justify-content: space-between; margin-top: 20px;">
                <div style="width: 48%; padding: 10px;">
                    <strong>Code Example:</strong><br>
                    <pre style="background-color: #2e2e2e; color: #f5f5f5; padding: 10px; border-radius: 5px; font-family: monospace;">{defect_row["code example"]}</pre>
                </div>
                <div style="width: 48%; padding: 10px;">
                    <strong>Code Fix Example:</strong><br>
                    <pre style="background-color: #2e2e2e; color: #f5f5f5; padding: 10px; border-radius: 5px; font-family: monospace;">{defect_row["code fix example"]}</pre>
                </div>
            </div>
        </div>
    </div>
    
    
    <div style="border: 1px solid #ccc; padding: 10px; margin-top: 20px; border-radius: 5px;">
        <strong>Example Submission:</strong><br>
        <pre style="background-color: #2e2e2e; color: #f5f5f5; padding: 10px; border-radius: 5px; font-family: monospace;">{submissions["answer"].iloc[random.randint(0, len(submissions) - 1)] if len(submissions) else 'No submissions found'}</pre>
    </div>
    """

In [ ]:
def gini(array):
    """Compute the Gini coefficient of a sorted numpy array."""
    array = array.flatten()
    # avoid zero division
    if array.sum() == 0:
        return 0.0
    # ensure all values are non-negative
    if np.amin(array) < 0:
        array -= np.amin(array)
    # order values for computation
    array = np.sort(array)
    index = np.arange(1, array.shape[0] + 1)
    # gini formula
    return ((np.sum((2 * index - array.shape[0] - 1) * array)) / (array.shape[0] * np.sum(array)))

***

# Loading data

In [ ]:
items = pd.read_csv(DATASET_PATH / f'items_{VERSION}.csv', index_col=0)
log = pd.read_csv(DATASET_PATH / f'log_{VERSION}.csv', index_col=0, parse_dates=['time'])
defects = pd.read_csv(DATASET_PATH / f'defects_{VERSION}.csv', index_col=0)
defect_log = pd.read_csv(DATASET_PATH / f'defect_log_{VERSION}.csv', index_col=0)
defect_log.columns = defect_log.columns.astype(int)
code_to_defect_id = json.load(open(DATASET_PATH / f'code_to_defect_id_{VERSION}.json', "r"))
defect_presence = defect_log > 0

defects['display name'] = defects['defect name'].apply(lambda x: abbreviate_and_truncate_text(x, ordered_abbreviations))
items['display name'] = items['name'].apply(truncate_text)

***

# Task filtering

## Task-defect pairs without minimal support

In [ ]:
insufficient_support = defect_presence.groupby(log["item"]).sum() < MIN_TASK_DEFECT_SUBMISSIONS
insufficient_support = insufficient_support.reindex(items.index).reindex(defects.index, axis=1).astype(bool)

***
# Data Split Analysis

TODO?

In [ ]:
def split_data_by_users(log, defect_log, pct=0.8):
    """
    Split the data into train and test sets, keeping submissions from the same user together.
    
    Expects log and defect log to share index.
    """
    all_users = log["user"].unique()
    np.random.seed(0)
    np.random.shuffle(all_users)

    train_users = all_users[:int(len(all_users) * pct)]
    mask = log["user"].isin(train_users)

    return log[mask], log[~mask], defect_log[mask], defect_log[~mask]

In [ ]:
train_log, test_log, train_defect_log, test_defect_log = split_data_by_users(log, defect_log)

In [ ]:
test_defects = test_defect_log > 0
print("Fraction of Submissions with Multiple Defects:", (test_defects.sum(axis=1) > 1).mean())
print("Total Number", (test_defects.sum(axis=1) > 1).sum())

***

# Heuristics

In [ ]:
data = items, defects

models = {
    "Task Common": TaskCommonModel(*data),
    "Task Characteristic": TaskCharacteristicModel(*data),
    "Currently Taught": CurrentlyTaughtPrioritizer(*data, data_path='data/currently_taught.txt'),
    "Student Frequency": StudentFrequencyModel(*data),
    "Student Characteristic": StudentCharacteristicModel(*data),
    "Student Encountered": StudentEncounteredBeforeModel(*data),
    "Defect Multiplicity": DefectMultiplicityModel(*data),
    "Severity Baseline": SeverityModel(*data),
}

In [ ]:
for name, model in (pbar :=tqdm(models.items(), desc="Training Models")):
    pbar.set_description(f"Training {name}")
    model.update(train_log, train_defect_log)

##  Weight Histograms

In [ ]:
def plot_model_weight_histogram(model_name, model, iamge_dir, bins=60):
    """Plot a histogram of model weights."""
    values = model.get_model_weights()

    if values is None:
        print(f"Model {model_name} has no weights. Skipping...")
        return

    values = values.values.flatten()

    fig, ax = plt.subplots(figsize=(12, 8), layout="constrained")

    n_unique = len(np.unique(values))
    discrete = n_unique <= 10

    sns.histplot(data=values, bins=bins if discrete else n_unique, ax=ax, discrete=discrete, shrink=0.8)
    
    ax.set_yscale('log')
    ax.set_xlabel(model.get_measure_name())
    ax.set_ylabel('Frequency (Log Scale)')
    ax.set_title(f'Distribution of {model.get_measure_description()}')
    
    plt.savefig(iamge_dir / model_name.lower().replace(" ", "_"), dpi=RESOLUTION)
    plt.close()

In [ ]:
temp_image_dir = IMAGE_DIR / 'model_weight_histograms'
os.makedirs(temp_image_dir, exist_ok=True)

for model_name, model in models.items():
    plot_model_weight_histogram(model_name, model, temp_image_dir)

## Task-Defect Weight Maps

In [ ]:
def plot_task_weight_heatmap(model_name, model, image_dir, defects, items, mask=None, normalize_cmap=False):
    """Plot a heatmap of task-defect model weights."""
    model_weights = model.get_model_weights().copy()

    if model_weights is None:
        print(f"Model {model_name} has no weights. Skipping...")
        return

    defect_names = defects['display name'].loc[model_weights.columns]
    task_names = items['display name'].loc[model_weights.index]

    fig, ax = plt.subplots(figsize=(12, 17), layout="constrained")
    
    if normalize_cmap:
        lim = max(np.abs(model_weights.values).max(), 1)
        kwargs = {'cmap': 'coolwarm', 'vmin':-lim, 'vmax':lim}
    else:
        kwargs = {'cmap': 'Reds'}

    model_weights[mask] = np.nan
        
    sns.heatmap(
        model_weights, xticklabels=defect_names, yticklabels=task_names, cbar=True,
        cbar_kws={'label': model.get_measure_name()}, **kwargs
    )

    ax.tick_params(axis='x', labelsize=7)
    ax.tick_params(axis='y', labelsize=8)
    plt.title(model.get_measure_description())
    plt.xlabel("Defects")
    plt.ylabel("Tasks")
    title = model_name.lower().replace(" ", "_")
    plt.savefig(image_dir / (title if mask is None else title + "_masked"), dpi=RESOLUTION)
    plt.close()

In [ ]:
temp_image_dir = IMAGE_DIR / 'task_defect_weight_maps'
os.makedirs(temp_image_dir, exist_ok=True)

for model_name, model in models.items():
    if model.get_context_type() == "task":
        normalize_cmap = model_name == "Task Characteristic"
        plot_task_weight_heatmap(model_name, model, temp_image_dir, defects, items, normalize_cmap=normalize_cmap)
    else:
        print(f"Model {model_name} is not a task model. Skipping...")

In [ ]:
temp_image_dir = IMAGE_DIR / 'task_defect_weight_maps_masked'
os.makedirs(temp_image_dir, exist_ok=True)

for model_name, model in models.items():
    if model.get_context_type() == "task":
        normalize_cmap = model_name == "Task Characteristic"
        plot_task_weight_heatmap(model_name, model, temp_image_dir, defects, items, normalize_cmap=normalize_cmap, mask=insufficient_support)
    else:
        print(f"Model {model_name} is not a task model. Skipping...")

## Quantitative analysis

In [ ]:
CLOSE_TIE_THRESHOLD = 0.05  # threshold for close ties as a percentage of the max score

metrics = {
    'exact_ties': {name: [] for name in models.keys()},
    'close_ties': {name: [] for name in models.keys()},
    'gini': {name: [] for name in models.keys()}
}

model_correlations = {
    name1: {name2: [] for name2 in models.keys()}
    for name1 in models.keys()
}

for _, submission in tqdm(test_log.iterrows(), total=test_log.shape[0]):
    defect_counts = test_defects.loc[submission.name]
    present_defects = defect_counts[defect_counts > 0]
    
    # Skip submissions with too few issues to prioritize
    if len(present_defects) < 2:
        for model in models.values():
            model.update(submission, defect_counts)
        continue

    prioritizations = []

    for name, model in models.items():
        scores = model.prioritize(submission, present_defects)
        
        # --- Exact Ties ---
        # Count items with the same score as the max
        max_score = scores.max()
        exact_ties = (scores == max_score).sum() > 1
        metrics['exact_ties'][name].append(exact_ties)

        # --- Close Ties ---
        # Count items within a relative threshold of the max score
        close_tie_threshold_value = max_score * CLOSE_TIE_THRESHOLD
        close_ties = (scores >= (max_score - close_tie_threshold_value)).sum() > 1
        metrics['close_ties'][name].append(close_ties)

        # --- Gini Coefficient ---
        metrics['gini'][name].append(gini(scores.values))

        # Store the prioritization to calculate intra-model agreement
        prioritizations.append(scores.values)

        # Update the model
        model.update(submission, defect_counts)
    
    # --- Agreement ---
    for i, first in enumerate(models.keys()):
        for j, second in enumerate(models.keys()):
            if i >= j:
                continue
            first_score = prioritizations[i]
            second_score = prioritizations[j]
            # Check for constant arrays
            if np.all(first_score == first_score[0]) or np.all(second_score == second_score[0]):
                rho = np.nan
            else:
                rho, _ = spearmanr(prioritizations[i], prioritizations[j])
            model_correlations[first][second].append(rho)
            model_correlations[second][first].append(rho)

# --- Final Aggregation and Formatting ---
results = {}
for metric_name, data in metrics.items():
    avg_values = {name: np.mean(values) for name, values in data.items()}
    results[f'avg_{metric_name}'] = pd.Series(avg_values)

print(f"Average Exact Ties: \n{results['avg_exact_ties']}")
print("-" * 20)
print(f"Average Close Ties: \n{results['avg_close_ties']}")
print("-" * 20)
print(f"Average Gini Coefficients: \n{results['avg_gini']}")

In [ ]:
json.dump(model_correlations, open('model_corelations.json', mode='w'))
json.dump(results, open('results.json', mode='w'))
json.dump(defects, open('defects.json', mode='w'))

### Decisivness - Ties

In [ ]:
plot_data = pd.DataFrame({
    'Exact Ties': results['avg_exact_ties'],
    'Close Ties': results['avg_close_ties'] - results['avg_exact_ties']
}).T.reset_index()

plot_data = plot_data.rename(columns={'index': 'Tie Type'})

plot_data_melted = plot_data.melt(id_vars='Tie Type', var_name='Model', value_name='Average Count')

ax = plot_data.set_index('Tie Type').T.plot(
    kind='bar', 
    stacked=True, 
    figsize=(12, 8),
    colormap='tab10'
)

plt.title("Average Number of Exact and Close Ties per Model", fontsize=16)
plt.ylabel("Average Count", fontsize=12)
plt.xlabel("Prioritization Model", fontsize=12)
plt.xticks(rotation=45, ha="right")

plt.legend(title='Tie Type')
plt.tight_layout()
plt.savefig(IMAGE_DIR / 'task_models_tie_bar_plot.png', dpi=RESOLUTION)
plt.close()

### Decisivness - Gini

In [ ]:
plot_data = []
for model_name, gini_list in metrics['gini'].items():
    for gini_value in gini_list:
        plot_data.append({'Model': model_name, 'Gini Coefficient': gini_value})

plot_data = pd.DataFrame(plot_data)

plt.figure(figsize=(12, 8))

sns.boxplot(
    x='Model',
    y='Gini Coefficient',
    data=plot_data,
    notch=True,
    palette='viridis',
    hue='Model',
    legend=False
)

plt.title("Distribution of Gini Coefficients by Prioritization Model", fontsize=16)
plt.xlabel("Prioritization Model", fontsize=12)
plt.ylabel("Gini Coefficient", fontsize=12)
plt.xticks(rotation=45, ha="right")
plt.ylim(0, 1)
plt.tight_layout()

plt.savefig(IMAGE_DIR / 'task_models_gini_box_plot.png', dpi=RESOLUTION)
plt.close()

### Inter-Model Agreement

In [ ]:
correlation_matrix = np.empty((len(task_models.keys()), len(task_models.keys())))
correlation_matrix[:] = np.nan

for i, first in enumerate(task_models.keys()):
    for j, second in enumerate(task_models.keys()):
        if i >= j:
            continue
        correlation_matrix[i, j] = np.nanmean(model_correlations[first][second])
        correlation_matrix[j, i] = correlation_matrix[i, j]

correlation_matrix = pd.DataFrame(correlation_matrix, index=task_models.keys(), columns=task_models.keys())

plt.figure(figsize=(10, 8))

sns.heatmap(
    correlation_matrix,
    annot=True,
    cmap='coolwarm',
    fmt=".2f",
    linewidths=.5,
    cbar_kws={'label': "Average Spearman's Correlation"},
    vmin=-1,
    vmax=1
)

plt.title("Inter-Model Prioritization Agreement (Spearman's Rho)", fontsize=16)
plt.xticks(rotation=45, ha="right")
plt.yticks(rotation=0)
plt.tight_layout()

plt.savefig(IMAGE_DIR / 'task_models_correlation_heatmap.png', dpi=RESOLUTION)
plt.close()

### Agreement with baseline

In [ ]:
plot_data = []
for model, corr_list in model_correlations['Severity Baseline'].items():
    for corr in corr_list:
        plot_data.append({'Model': model, 'Spearman_Rho': corr})

plot_data = pd.DataFrame(plot_data)

plt.figure(figsize=(10, 8))

sns.boxplot(
    x='Model',
    y='Spearman_Rho',
    data=plot_data,
    palette='rocket',
    hue='Model',
    legend=False,
    notch=True
)

plt.title("Distribution of Spearman's Rho by Prioritization Model", fontsize=16)
plt.xlabel("Prioritization Model", fontsize=12)
plt.ylabel("Spearman's Rho", fontsize=12)
plt.xticks(rotation=45, ha="right")
plt.tight_layout()

plt.savefig(IMAGE_DIR / 'task_models_baseline_correlation_box_plot.png', dpi=RESOLUTION)
plt.close()

### Sensitivity

In [ ]:
matrices = {}
for name, model in tqdm(models.items(), desc="Analyzing Task Models"):
    if model.get_context_type() != "task":
        continue
    
    weights = model.get_model_weights()

    correlation_matrix = pd.DataFrame(index=items.index, columns=items.index)

    if weights is None:
        matrix_values = np.ones((len(items.index), len(items.index)))
        np.fill_diagonal(matrix_values, 0)
        correlation_matrix = pd.DataFrame(matrix_values, index=items.index, columns=items.index)
        matrices[name] = correlation_matrix
        continue

    for first in items.index:
        for second in items.index:
            if first >= second:
                continue
            weights_first = weights.loc[first]
            weights_second = weights.loc[second]

            if np.all(weights_first == weights_first.iloc[0]) or np.all(weights_second == weights_second.iloc[0]):
                rho = np.nan
            else:
                rho, _ = spearmanr(weights_first, weights_second)

            correlation_matrix.loc[first, second] = rho
            correlation_matrix.loc[second, first] = rho
    matrices[name] = correlation_matrix

In [ ]:
fig, axes = plt.subplots(
    nrows=2, ncols=2, figsize=(20, 18),
    sharex=True, sharey=True,
    gridspec_kw={'wspace': 0.05, 'hspace': 0.05} # Fine-tune the spacing
)
axes = axes.flatten()

models = ["Task Common", "Task Characteristic", "Currently Taught", "Severity Baseline"]
cmaps = ['coolwarm', 'coolwarm', 'coolwarm', 'coolwarm']

for i, name in enumerate(models):
    ax = axes[i]
    correlation_matrix = matrices[name]
    
    sns.heatmap(
        correlation_matrix.astype(float),
        ax=ax,
        cmap=cmaps[i],
        vmin=-1,
        vmax=1,
        linewidths=.5,
        cbar=False,
    )

    ax.set_title(f"Task-to-Task Agreement for {name}", fontsize=18)
    if i in [2, 3]:
        ax.set_xlabel('Task ID', fontsize=12)
    else:
        ax.set_xlabel('')
        ax.set_xticklabels([])
    if i in [0, 2]:
        ax.set_ylabel('Task ID', fontsize=12)
    else:
        ax.set_ylabel('')
        ax.set_yticklabels([])

# Create a single color bar for the entire figure
fig.subplots_adjust(right=0.85)
cbar_ax = fig.add_axes([0.88, 0.15, 0.02, 0.7])
last_heatmap = axes[-1].collections[0]
cbar = fig.colorbar(last_heatmap, cax=cbar_ax)
cbar.set_label("Spearman's Correlation", fontsize=16)

plt.savefig(IMAGE_DIR / 'task_to_task_correlation_heatmaps.png', dpi=RESOLUTION)
plt.close()

In [ ]:
std_devs = {}
for name, correlation_matrix in matrices.items():
    upper_triangle_mask = np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool)
    off_diagonal_values = correlation_matrix.where(upper_triangle_mask).stack().values
    off_diagonal_std = off_diagonal_values.std()
    
    std_devs[name] = off_diagonal_std

std_df = pd.DataFrame(std_devs.items(), columns=['Model', 'Standard Deviation'])

plt.figure(figsize=(10, 6))
sns.barplot(x='Model', y='Standard Deviation', data=std_df, palette='viridis', hue='Model', legend=False)

plt.title("Standard Deviation of Inter-Task Correlation", fontsize=16)
plt.ylabel("Standard Deviation of Spearman's Rho", fontsize=12)
plt.xlabel("Prioritization Model", fontsize=12)
plt.xticks(rotation=45, ha="right")
plt.tight_layout()

plt.savefig(IMAGE_DIR / 'task_models_sensitivity_bar_plot.png', dpi=RESOLUTION)
plt.close()

# Pilot Testing

# Output